In [2]:
#VARIABLES to set up
# SETUP the name of the file where result will be write
SF = 10 #choose between 1,3,5,8,10


In [ ]:
#import
import os
import time
from pyspark.sql import SparkSession
import glob

# Initialize SparkSession
spark = SparkSession.builder.appName("TPC-DS Data Loading").config("spark.sql.catalogImplementation", "hive").getOrCreate()

In [ ]:
#create all tables
with open(os.getcwd() + "/ddl/Create_database.sql", 'r') as file: # path/to/tpcds.sql
    db_string = file.read()

db_string = str.split(db_string, ";")
db_string = db_string[:-1]

for string in db_string:
    if string != '\n' or string != '':
        spark.sql(string)

In [ ]:
#load datas into tables
table_loading =[]
data_file = glob.glob(os.getcwd() + "/data/data"+ str(SF)+"/*")
for i in data_file:
    table_name = i.split("/")[-1].split(".")[0]
    if table_name != "dbgen_version":
        sql_command = "select * from " + table_name
        tmp=spark.sql(sql_command)
        #print(tmp)
        table_load_start = time.time()
        df = spark.read.schema(tmp.schema).csv( i, sep='|')
        df.write.mode("append").insertInto(table_name)
        table_load_end = time.time()
        table_loading.append([table_name, table_load_end - table_load_start])
        print("Data from table " + table_name + " is inserted")
        
        
#write result        
f = open("result/table_time_SF"+ str(SF) + ".csv", "w")
f.write( str(table_loading) +'\n')
f.close()

In [55]:
#run queries
queries_files = glob.glob(os.getcwd() + "/Queries/query*")
j = 0
time1=[0 for i in range(99)]
time2=[0 for i in range(99)]
time3=[0 for i in range(99)]
time4=[0 for i in range(99)]
time5=[0 for i in range(99)]

for i in queries_files:
    with open( i , 'r') as file:
        db_string = file.read()
    db_string=db_string.split(";")[:-1]
    #start time
    number_queries = int(i.split("query")[-1].split(".")[0])
    for i in db_string:
        try:
            j+=1
            exec_start = time.time()
            spark.sql(i)
            exec_end = time.time()
            time1[number_queries-1] += exec_end- exec_start
            exec_start = time.time()
            spark.sql(i)
            exec_end = time.time()
            time2[number_queries-1] += exec_end- exec_start
            exec_start = time.time()
            spark.sql(i)
            exec_end = time.time()
            time3[number_queries-1] += exec_end- exec_start
            exec_start = time.time()
            spark.sql(i)
            exec_end = time.time()
            time4[number_queries-1] += exec_end- exec_start
            exec_start = time.time()
            spark.sql(i)
            exec_end = time.time()
            time5[number_queries-1] += exec_end- exec_start
            
        except:
            print(str(number_queries), " yes")


    
    #end time
f = open("result/sf"+ str(SF) + ".csv", "w")
f.write(str(time1)+ '\n' + str(time2)+ '\n'+ str(time3)+ '\n'+ str(time4)+ '\n' + str(time5))
f.close()

In [14]:
#Throughput TEST 1
time_trought=[0,0,0,0,0,0,0,0]

for i in range(1,5):
    queries_files = glob.glob(os.getcwd() + "/Query_1-4/query*_stream"+ str(i)+"*")
    
    for j in queries_files:
        with open( j , 'r') as file:
            db_string = file.read()
            
            
        db_string=db_string.split(";")[:-1]
        for k in db_string:
            exec_start = time.time()
            spark.sql(k)
            exec_end = time.time()
            time_trought[i-1] += exec_end- exec_start
            
#some maintenance to do...            
            
# ???
    
    
#some new THrought test 2
for i in range(5,9):
    queries_files = glob.glob(os.getcwd() + "/Query_5-8/query*_stream"+ str(i)+"*")
    
    for j in queries_files:
        with open( j , 'r') as file:
            db_string = file.read()
            
            
        db_string=db_string.split(";")[:-1]
        for k in db_string:
            exec_start = time.time()
            spark.sql(k)
            exec_end = time.time()
            time_trought[i-1] += exec_end- exec_start
print(time_trought)

f = open("result/ThroughTestsf"+ str(SF) + ".csv", "w")
f.write(str(time_trought))
f.close()

[5.831288814544678, 5.224609613418579, 5.151582717895508, 5.2003748416900635, 5.193056583404541, 5.185740947723389, 5.209424257278442, 5.155352592468262]


In [10]:

#maintenace type 2
mt2=[["DF_SS","store_sales","store_returns"],["DF_CS", "catalog_sales","catalog_returns"],["DF_WS", "web_sales","web_returns"]]
for i in mt2:
    query1 = "Delete rows from " +i[]+ "with corresponding rows in" + S +"where d_date between"+ Date1 +"and"+ Date2
    query2 = "Delete rows from"+ S +"where d_date between" + Date1 +"and"+ Date2
#mainteance type 3
["DF_I", "Inventory"]

"Delete rows from Inventory where d_date between " + Date1 + " and "+ Date2
